In [1]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.types import IntegerType, ShortType

In [2]:
databricks = False
overwrite  = False
is_yellow  = False
yellow     = "yellow" if is_yellow else "foil"

In [3]:
if not databricks:
    data_dir = "../data"
    spark = SparkSession.builder.getOrCreate()
else:
    data_dir = "/dbfs/mnt/group01"

raw_pquet = raw_dbfs = data_dir + "/{}".format(yellow) +"/raw/{}/{}.gz.parquet"
cln_pquet = cln_dbfs = data_dir + "/{}".format(yellow) +"/raw-cln/{}/{}.gz.parquet"

if databricks:
    raw_dbfs = raw_dbfs.replace("/dbfs", "")
    cln_dbfs = cln_dbfs.replace("/dbfs", "")

In [4]:
if is_yellow:
    start_year = 2009
    end_year   = 2016
else:
    start_year = 2010
    end_year   = 2013

In [5]:
year_range = range(start_year, end_year + 1)
month_range = range(1, 13)

In [6]:
w_lon = -74.2463  # left bound
e_lon = -73.7141  # right bound
n_lat = 40.9166  # up bound
s_lat = 40.4767  # down bound

In [7]:
def check_file_exist(_path):
    if os.path.exists(_path) and not overwrite:
            print("[SYSTEM]: File exists: {}".format(_path))
            return True
    else:
        return False

In [8]:
def filter_by_bound(_in_df):
    return _in_df.filter(
        (e_lon >= func.col("pickup_longitude"))  &
        (w_lon <= func.col("pickup_longitude"))  &
        (e_lon >= func.col("dropoff_longitude")) &
        (w_lon <= func.col("dropoff_longitude")) &
        (n_lat >= func.col("pickup_latitude"))   &
        (s_lat <= func.col("pickup_latitude"))   &
        (n_lat >= func.col("dropoff_latitude"))  &
        (s_lat <= func.col("dropoff_latitude"))
    )

In [9]:
def round_coordinate(_in_df):
    return _in_df\
        .withColumn("dropoff_latitude",  func.round(_in_df["dropoff_latitude"], 4)) \
        .withColumn("dropoff_longitude", func.round(_in_df["dropoff_longitude"], 4)) \
        .withColumn("pickup_latitude",   func.round(_in_df["pickup_latitude"], 4)) \
        .withColumn("pickup_longitude",  func.round(_in_df["pickup_longitude"], 4))

In [10]:
def filter_duration(_in_df):
    return _in_df.withColumn("duration_second",
                      (func.unix_timestamp(func.col("dropoff_datetime")) -
                       func.unix_timestamp(func.col("pickup_datetime")))
                      .cast(IntegerType())
                      )\
        .filter(func.col("duration_second") > 45)

In [11]:
def filter_distance(_in_df):
    return _in_df.filter(func.col("trip_distance") > 0.2)

In [12]:
def expand_time(_in_df):
    return _in_df\
        .withColumn("drop_min", func.minute("dropoff_datetime")) \
        .withColumn("drop_hour", func.hour("dropoff_datetime")) \
        .withColumn("drop_day", func.dayofmonth("dropoff_datetime"))\
        .withColumn("drop_month", func.month("dropoff_datetime")) \
        .withColumn("drop_year", func.year("dropoff_datetime")) \
        .withColumn("pick_min", func.minute("pickup_datetime"))\
        .withColumn("pick_hour", func.hour("pickup_datetime")) \
        .withColumn("pick_day", func.dayofmonth("pickup_datetime")) \
        .withColumn("pick_month", func.month("pickup_datetime")) \
        .withColumn("pick_year", func.year("pickup_datetime")) \
        .withColumn("week_day", func.dayofweek("pickup_datetime")) \

In [14]:
def update_schema(_in_df):
    for c in _in_df.columns:
        if ("_hour" in c) or ("_day" in c) or ("_month" in c) or ("season" in c):
            _in_df = _in_df.withColumn(c, func.col(c).cast(ShortType()))
        return _in_df
def select_columns(_in_df):
    if not is_yellow:
        _in_df = _in_df.drop("trip_time_in_secs")
    else:
        for c in _in_df.columns:
            if c in ["vendor_id", "passenger_count", "store_and_forward", "payment_type",
                     "fare_amount", "surcharge", "mta_tax", "tolls_amount"]:
                _in_df = _in_df.drop(c)
    return _in_df

In [15]:
def process():
    info_title = lambda _mode, _y, _m : print("____________________________{}_PROCESS_{}_{}____________________________".format(_mode.upper(), _y, _m))
    info_start = lambda _y, _m : print("[SYSTEM]: Start  {}-{}".format(_y, _m))
    info_end   = lambda _y, _m : print("[SYSTEM]: Finish {}-{}".format(_y, _m))
    for year in year_range:
        for month in month_range:
            if not os.path.exists(raw_pquet.format(year, month)):
                continue
            info_title(yellow, year, month)
            if check_file_exist(cln_pquet.format(year, month)):
                continue
            info_start(year, month)
            df = spark.read.parquet(raw_dbfs.format(year, month))
            # 1. Filter by longitude and latitude
            df = filter_by_bound(df)
            # # Process time/length
            # # 1. Date time to day/month/hour/minutes
            # # 2. Get duration in seconds
            # # 3. Remove trip duration less than 45 seconds
            # # 3. Remove trip distance less than 200 meters
            df = filter_duration(df)
            df = filter_distance(df)

            # 1. Round to four digit
            df = round_coordinate(df)
            df = expand_time(df)
            df = expand_season(df, month)
            df = update_schema(df)
            df = select_columns(df)
            df.repartition("pick_day", "pick_hour")\
                .write.mode("overwrite")\
                .option("compression", "gzip")\
                .partitionBy("pick_day", "pick_hour")\
                .parquet(cln_dbfs.format(year, month))
            info_end(year, month)

In [16]:
process()

____________________________FOIL_PROCESS_2010_1____________________________
[SYSTEM]: File exists: ../data/foil/raw-cln/2010/1.gz.parquet
____________________________FOIL_PROCESS_2010_2____________________________
[SYSTEM]: Start  2010-2
[SYSTEM]: Finish 2010-2
____________________________FOIL_PROCESS_2010_3____________________________
[SYSTEM]: Start  2010-3
[SYSTEM]: Finish 2010-3
____________________________FOIL_PROCESS_2010_4____________________________
[SYSTEM]: Start  2010-4
[SYSTEM]: Finish 2010-4
____________________________FOIL_PROCESS_2010_5____________________________
[SYSTEM]: Start  2010-5
[SYSTEM]: Finish 2010-5
____________________________FOIL_PROCESS_2010_6____________________________
[SYSTEM]: Start  2010-6


KeyboardInterrupt: 

In [ ]:
# root-foil-raw
#  |-- medallion: integer (nullable = true)
#  |-- hack_license: integer (nullable = true)
#  |-- pickup_datetime: timestamp (nullable = true)
#  |-- dropoff_latitude: double (nullable = true)
#  |-- dropoff_longitude: double (nullable = true)
#  |-- pickup_latitude: double (nullable = true)
#  |-- pickup_longitude: double (nullable = true)
#  |-- trip_distance: double (nullable = true)
#  |-- trip_time_in_secs: integer (nullable = true)
#  |-- dropoff_datetime: timestamp (nullable = true)
#  |-- rate_code: short (nullable = true)
#  |-- tip_amount: double (nullable = true)
#  |-- total_amount: double (nullable = true)

# root-yellow-raw
#  |-- vendor_id: double (nullable = true)
#  |-- pickup_datetime: timestamp (nullable = true)
#  |-- dropoff_datetime: timestamp (nullable = true)
#  |-- passenger_count: double (nullable = true)
#  |-- trip_distance: double (nullable = true)
#  |-- pickup_longitude: double (nullable = true)
#  |-- pickup_latitude: double (nullable = true)
#  |-- rate_code: double (nullable = true)
#  |-- store_and_forward: double (nullable = true)
#  |-- dropoff_longitude: double (nullable = true)
#  |-- dropoff_latitude: double (nullable = true)
#  |-- payment_type: double (nullable = true)
#  |-- fare_amount: double (nullable = true)
#  |-- surcharge: double (nullable = true)
#  |-- mta_tax: double (nullable = true)
#  |-- tip_amount: double (nullable = true)
#  |-- tolls_amount: double (nullable = true)
#  |-- total_amount: double (nullable = true)

# root
#  |-- pickup_datetime: timestamp (nullable = true)
#  |-- dropoff_datetime: timestamp (nullable = true)
#  |-- trip_distance: double (nullable = true)
#  |-- pickup_longitude: double (nullable = true)
#  |-- pickup_latitude: double (nullable = true)
#  |-- rate_code: double (nullable = true)
#  |-- dropoff_longitude: double (nullable = true)
#  |-- dropoff_latitude: double (nullable = true)
#  |-- tip_amount: double (nullable = true)
#  |-- total_amount: double (nullable = true)
#  |-- duration_second: integer (nullable = true)
#  |-- drop_day: integer (nullable = true)
#  |-- drop_month: integer (nullable = true)
#  |-- drop_hour: integer (nullable = true)
#  |-- drop_min: integer (nullable = true)
#  |-- week_day: integer (nullable = true)
#  |-- pick_month: integer (nullable = true)
#  |-- pick_min: integer (nullable = true)
#  |-- season: integer (nullable = true)
#  |-- pick_day: integer (nullable = true)
#  |-- pick_hour: integer (nullable = true)
#
# root
#  |-- medallion: integer (nullable = true)
#  |-- hack_license: integer (nullable = true)
#  |-- pickup_datetime: timestamp (nullable = true)
#  |-- dropoff_latitude: double (nullable = true)
#  |-- dropoff_longitude: double (nullable = true)
#  |-- pickup_latitude: double (nullable = true)
#  |-- pickup_longitude: double (nullable = true)
#  |-- trip_distance: double (nullable = true)
#  |-- dropoff_datetime: timestamp (nullable = true)
#  |-- rate_code: short (nullable = true)
#  |-- tip_amount: double (nullable = true)
#  |-- total_amount: double (nullable = true)
#  |-- duration_second: integer (nullable = true)
#  |-- drop_day: integer (nullable = true)
#  |-- drop_month: integer (nullable = true)
#  |-- drop_hour: integer (nullable = true)
#  |-- drop_min: integer (nullable = true)
#  |-- week_day: integer (nullable = true)
#  |-- pick_month: integer (nullable = true)
#  |-- pick_min: integer (nullable = true)
#  |-- season: integer (nullable = true)
#  |-- pick_day: integer (nullable = true)
#  |-- pick_hour: integer (nullable = true)